<center><img src="./logo.png" alt="Header" style="width: 800px;"/></center>

In [ ]:
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
from Arm_Lib import Arm_Device

Arm = Arm_Device()


In [ ]:
import traitlets
import ipywidgets.widgets as widgets
import time
import threading
import inspect
import ctypes

################################################################################################################################################################################################################################################################
#IK
################################################################################################################################################################################################################################################################

# import the needed modules (as is used to rename the module for easier referencing)
import math
import numpy as np
import general_robotics_toolbox as rox

import numpy as np
import general_robotics_toolbox as rox #ik
import time
import math
from Arm_Lib import Arm_Device #
import math
import numpy as np
import general_robotics_toolbox as rox

# define the Jacobian inverse 
def jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol):
# the inputs are 
	# robot: of the Robot class defined in rox. contains rotation axes, position vectors, joint types
	# q0: the initial guess joint angles as a 5x1 numpy array
	# Rd: the desired 3x3 rotation matrix (R_{0T}) as a numpy array
	# Pd: the desired 3x1 position vector (P_{0T}) as a numpy array
	# Nmax: the maximum number of allowed iterations
	# alpha: the update parameter
	# tol: the tolerances for [roll, pitch, yaw, x, y, z] as a 6x1 numpy array

# set up storage space
    n = len(q0)
    q = np.zeros((n,Nmax+1))
    q[:,0] = q0
    p0T = np.zeros((3,Nmax+1))
    RPY0T = np.zeros((3,Nmax+1))
    iternum = 0

# compute the forward kinematics
    H = rox.fwdkin(robot,q[:,0])
    R = H.R
    P = H.p
    P = np.array([[P[0]], [P[1]], [P[2]]])

# get the initial error
    dR = np.matmul(R, np.transpose(Rd))
    r = np.array(rox.R2rpy(dR))[None]
    dX = np.concatenate((np.transpose(r), P-Pd))

# iterate while any error element is greater than its tolerance
    while (np.absolute(dX) > tol).any():
	# stop execution if the maximum number of iterations is exceeded
        if iternum < Nmax:
		# compute the forward kinematics
            H = rox.fwdkin(robot, q[:,iternum])
            R = H.R
            p0T = H.p
            p0T = np.array([[p0T[0]], [p0T[1]], [p0T[2]]])
		# compute the error
            dR = np.matmul(R , np.transpose(Rd))
            r = np.array(rox.R2rpy(dR))[None]
            dX = np.concatenate((np.transpose(r), p0T-Pd))

		# calculate the Jacobian matrix
            Jq = rox.robotjacobian(robot, q[:, iternum])
		# compute the update
            j = np.matmul(np.linalg.pinv(Jq), dX)
		# use the update to generate a new q
            q[:, iternum+1] = q[:, iternum] - np.transpose((alpha * j))
            iternum = iternum + 1
        else:
            break
	# return the final estimate of q
    return q[:, iternum]


def rotation_matrix_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

def rotation_matrix_y(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def rotation_matrix_x(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def calculate_rotation_matrix(angles):
    q1, q2, q3, q4, q5 = np.radians(angles)  # Convert degrees to radians
    R01 = rotation_matrix_z(q1)
    R12 = rotation_matrix_y(-q2)
    R23 = rotation_matrix_y(-q3)
    R34 = rotation_matrix_y(-q4)
    R45 = rotation_matrix_x(-q5)
    R0T = R01 @ R12 @ R23 @ R34 @ R45  # Matrix multiplication
    return R0T

def return_rotations(angles):
    q1, q2, q3, q4, q5 = np.radians(angles)  # Convert degrees to radians
    R01 = rotation_matrix_z(q1)
    R12 = rotation_matrix_y(-q2)
    R23 = rotation_matrix_y(-q3)
    R34 = rotation_matrix_y(-q4)
    R45 = rotation_matrix_x(-q5)
    rotations = [R01, R12, R23, R34, R45]
    return rotations

def return_pd(rotations, points):
    pd = points[0]+rotations[0] @ rotations[1] @ (points[1]+rotations[2] @ (points[2]+rotations[3] @ rotations[4] @ points[3])) 
    return pd


def dofik(a1, a2, a3, a4, a5, a6):
   
    tol = np.array(np.transpose([[0.02, 0.02, 0.02, 0.001, 0.001, 0.001]]))
    Nmax = 200
    alpha = 0.1

    # Define all the joint lengths [m]
    l0 = 61 * 10**-3
    l1 = 43.5 * 10**-3
    l2 = 82.85 * 10**-3
    l3 = 82.85 * 10**-3
    l4 = 73.85 * 10**-3
    l5 = 54.57 * 10**-3

	# define the unit vectors
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])

    # Define the position vectors from i-1 -> i
    P01 = (l0 + l1) * ez
    P12 = np.zeros(3)
    P23 = l2 * ex
    P34 = -1*l3 * ez
    P45 = np.zeros(3)
    P5T = -1*(l4 + l5) * ex
    
    points = [P01, P23, P34, P5T]
    
	# define the class inputs: rotation axes (H), position vectors (P), and joint_type
    H = np.array([ez, -1*ey, -1*ey, -1*ey, -1*ex]).T
    P = np.array([P01, P12, P23, P34, P45, P5T]).T
    joint_type = [0,0,0,0,0]
    
   

	# define the Robot class
    robot = rox.Robot(H, P, joint_type)
	
    s_time=3000
    qd=[a1, a2, a3, a4, a5]
    q0 = np.array(np.transpose([a1-5, a2-5, a3-5, a4-5, a5-5]))*math.pi/180
    Rd = calculate_rotation_matrix(qd)
    Pd = return_pd(return_rotations(qd), points)
    Pd_array = np.array(Pd)
    Pd = Pd_array.reshape(3, 1)
    
    # compute the inverse kinematics
    q = jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol)
	# convert solution to degrees
    q = q * 180 / math.pi
    print(q) #this gives an array of q values for the end effector. Could be simply inputted to servo commands like we used for fk. Arm.Arm_serial_servo_write6(angle, angle, angle, angle, angle, angle-90, s_time) #FIXME
    Arm.Arm_serial_servo_write6(q[0], q[1], q[2], q[3], q[4], a6, s_time) #FIXME
    
angle = 90

# def cupstack_to_dispenser():
#     dofik(angle, angle, angle, angle, angle, angle) 
#     time.sleep(1.5)
#     dofik(angle-45, angle+60, angle-150, angle+5, angle, angle) 
#     time.sleep(1.5)
#     dofik(angle-45, angle+60, angle-150, angle+5, angle, angle+57) 
#     time.sleep(1.5)
#     dofik(angle-45, angle+45, angle-150, angle+45, angle, angle+57) 
#     time.sleep(1.5)
#     dofik(angle-26, angle+45, angle-150, angle+45, angle+180, angle+57) 
#     time.sleep(1.5)
#     dofik(angle-26, angle-90, angle-70, angle+80, angle+180, angle+57) 
#     time.sleep(1.5)
#     dofik(angle-26, angle-90, angle-70, angle+80, angle+180, angle) 
#     time.sleep(1.5)

# cupstack_to_dispenser()

# def dispenser_press():
#     dofik(angle-26, angle-45, angle-70, angle+80, angle+180, angle)
#     time.sleep(1.5)
#     dofik(angle-26, angle-45, angle-70, angle+80, angle, angle) 
#     time.sleep(1.5)
#     dofik(angle-26, angle-45, angle-20, angle-15, angle, angle+89) 
#     time.sleep(1.5)
#     dofik(angle-26, angle-45, angle-20, angle-25, angle, angle+89) 
#     time.sleep(1.5)
#     dofik(angle-26, angle-45, angle-20, angle-7, angle, angle+89) 
# dispenser_press()

# def dispenser_to_cup():
#     time.sleep(1.5)
#     dofik(angle-26, angle-45, angle-70, angle+85, angle, angle) 
#     time.sleep(1.5)
#     dofik(angle-26, angle-90, angle-70, angle+85, angle, angle)
#     time.sleep(1.5)
#     dofik(angle-26, angle-90, angle-70, angle+85, angle, angle+57)
# dispenser_to_cup()

# def dispenser_to_patron():
#     time.sleep(1.5)
#     dofik(angle+26, angle-90, angle-70, angle+85, angle, angle+57)
#     time.sleep(1.5)
#     dofik(angle+26, angle-90, angle-70, angle+85, angle, angle)
# dispenser_to_patron()
    
print("passed IK")

origin_widget = widgets.Image(format='jpeg', width=320, height=240)
mask_widget = widgets.Image(format='jpeg',width=320, height=240)
result_widget = widgets.Image(format='jpeg',width=320, height=240)

image_container = widgets.HBox([origin_widget, mask_widget, result_widget])
# image_container = widgets.Image(format='jpeg', width=600, height=500)
display(image_container)

In [ ]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [ ]:
def get_color(img):
    H = []
    color_name={}
    img = cv2.resize(img, (640, 480), )
    HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    cv2.rectangle(img, (280, 180), (360, 260), (0, 255, 0), 2)
    for i in range(280, 360):
        for j in range(180, 260): H.append(HSV[j, i][0])
    H_min = min(H);H_max = max(H)
#     print(H_min,H_max)
    if H_min >= 0 and H_max <= 10 or H_min >= 156 and H_max <= 180: color_name['name'] = 'red'
    elif H_min >= 26 and H_max <= 34: color_name['name'] = 'yellow'
    elif H_min >= 35 and H_max <= 78: color_name['name'] = 'green'
    elif H_min >= 100 and H_max <= 124: color_name['name'] = 'blue'
    return img, color_name

In [ ]:
# Define variable parameters at different locations
# 定义不同位置的变量参数
look_at = [90, 164, 18, 0, 90, 90]
p_top = [90, 80, 50, 50, 270]

p_Yellow = [65, 22, 64, 56, 270]
p_Red = [118, 19, 66, 56, 270]

p_Green = [136, 66, 20, 29, 270] #[136, 66, 20, 29, 270]
p_Blue = [44, 66, 20, 28, 270]

p_gray = [90, 48, 35, 30, 270]



In [ ]:
# Define control DOFBOT function, control No.1-No.6 servo，p=[S1,S2,S3,S4,S5,S6]
# 定义移动机械臂函数,同时控制1-6号舵机运动，p=[S1,S2,S3,S4,S5,S6]
def arm_move_6(p, s_time = 500):
    for i in range(6):
        id = i + 1
        Arm.Arm_serial_servo_write(id, p[i], s_time)
        time.sleep(.01)
    time.sleep(s_time/1000)
    
# Define control DOFBOT function, control No.1-No.5 servo，p=[S1,S2,S3,S4,S5]
# 定义移动机械臂函数,同时控制1-5号舵机运动，p=[S1,S2,S3,S4,S5]
def arm_move(p, s_time = 500):
    for i in range(5):
        id = i + 1
        if id == 5:
            time.sleep(.1)
            Arm.Arm_serial_servo_write(id, p[i], int(s_time*1.2))
        elif id == 1 :
            Arm.Arm_serial_servo_write(id, p[i], int(3*s_time/4))
        else:
            Arm.Arm_serial_servo_write(id, p[i], int(s_time))
        time.sleep(.01)
    time.sleep(s_time/1000)
    
# enable=1：clip，=0：release
def arm_clamp_block(enable):
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 60, 400)
    else:
        Arm.Arm_serial_servo_write(6, 130, 400)
    time.sleep(.5)

In [ ]:
arm_move_6(look_at, 1000)
time.sleep(1)

In [ ]:
global g_state_arm
g_state_arm = 0

def ctrl_arm_move(index):
    arm_clamp_block(0)
    if index == 1:
        print("Yellow")
        number_action(index)
        put_down_block()
    elif index == 2:
        print("Red")
        number_action(index)
        put_down_block()
        print("Here")
    elif index == 3:
        print("Green")
        number_action(index)
        put_down_block()
    elif index == 4:
        print("Blue")
        number_action(index)
        put_down_block()
    
    global g_state_arm
    g_state_arm = 0


In [ ]:
# #Define the function corresponding to each number
# 数字功能定义
def number_action(index):
    if index == 1:
        # Grab yellow block
        # 抓取黄色的积木块
        dofik(90, 164, 18, 0, 90, 90)
        dofik(90, 164, 18, 0, 90, 90)
        dofik(angle-27, 164, 18, 0, 90, 90)
        time.sleep(2.5)
        dofik(angle-27, angle-90, angle-70, angle+85, angle, angle)
        time.sleep(2.5)
        dofik(angle-27, angle-90, angle-70, angle+85, angle, angle+30)
        time.sleep(2.5)
        time.sleep(3.0)
        dofik(angle-15, angle-60, angle-70, angle+50, angle, angle+30)
        time.sleep(3.0)
        dofik(angle-12, angle-40, angle-107, angle+55, angle, angle+30)
        time.sleep(2.25)
        dofik(angle-8, angle-37, angle-107, angle+55, angle-90, angle+30)
        
        #arm_move(p_top, 1000)
    elif index == 2:
        time.sleep(.5)
        # Grab red block
            # 抓取红色的积木块   

        dofik(90, 164, 18, 0, 90, 90)
        dofik(angle+23, 164, 18, 0, 90, 90)
        time.sleep(2.5)
        dofik(angle+23, angle-90, angle-70, angle+85, angle, angle)
        time.sleep(2.5)
        dofik(angle+23, angle-90, angle-70, angle+85, angle, angle+30)
        time.sleep(2.5)
        
        time.sleep(3.0)
        dofik(angle+10, angle-60, angle-70, angle+50, angle, angle+30)
        time.sleep(3.0)
        dofik(angle+8, angle-40, angle-107, angle+55, angle, angle+30)
        time.sleep(2.25)
        dofik(angle+3, angle-37, angle-107, angle+55, angle+90, angle+30)
     
    elif index == 3:
        # Grab green block
        # 抓取绿色的积木块
        arm_move(p_top, 1000)
        arm_move(p_Green, 1000)
        arm_clamp_block(1)
        arm_move(p_top, 1000)
    elif index == 4:
        # Grab blue block
        # 抓取蓝色的积木块
        arm_move(p_top, 1000)
        arm_move(p_Blue, 1000)
        arm_clamp_block(1)
        arm_move(p_top, 1000)

    
def put_down_block():
    return 0

    

In [ ]:
def start_move_arm(index):
    global g_state_arm
    if g_state_arm == 0:
        closeTid = threading.Thread(target = ctrl_arm_move, args = [index])
        closeTid.setDaemon(True)
        closeTid.start()
        
        g_state_arm = 1

# 主进程

# Main process

In [ ]:
import cv2
import numpy as np
import ipywidgets.widgets as widgets


cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(5, 30)  
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# image.set(cv2.CAP_PROP_BRIGHTNESS, 40) 
# image.set(cv2.CAP_PROP_CONTRAST, 50)   
# image.set(cv2.CAP_PROP_EXPOSURE, 156)  

# Red is selected by default, and the program will automatically switch the color according to the color detected in the box
# 默认选择红色的,程序会自动根据方框中检测到的颜色切换颜色
# Red value
# 红色区间
color_lower = np.array([0, 43, 46])
color_upper = np.array([10, 255, 255])

# green value
# 绿色区间
# color_lower = np.array([35, 43, 46])
# color_upper = np.array([77, 255, 255])

# blue value
# #蓝色区间
# color_lower=np.array([100, 43, 46])
# color_upper = np.array([124, 255, 255])

# yellow value
# #黄色区间
# color_lower = np.array([26, 43, 46])
# color_upper = np.array([34, 255, 255])

# orange value
# #橙色区间
# color_lower = np.array([11, 43, 46])
# color_upper = np.array([25, 255, 255])


def Color_Recongnize():
    Arm.Arm_Buzzer_On(1)
    s_time = 300
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    
    while(1):
        ret, frame = cap.read()
        frame, color_name = get_color(frame)
        if len(color_name)==1:
            global color_lower
            global color_upper
#             print ("color_name :", color_name)
#             print ("name :", color_name['name'])
            if color_name['name'] == 'yellow':
                color_lower = np.array([26, 43, 46])
                color_upper = np.array([34, 255, 255])
                start_move_arm(1)
            elif color_name['name'] == 'red':
                color_lower = np.array([0, 43, 46])
                color_upper = np.array([10, 255, 255])
                start_move_arm(2)
            elif  color_name['name'] == 'green':
                color_lower = np.array([35, 43, 46])
                color_upper = np.array([77, 255, 255])
                start_move_arm(3)
            elif color_name['name'] == 'blue':
                color_lower=np.array([100, 43, 46])
                color_upper = np.array([124, 255, 255])
                start_move_arm(4)
            

        origin_widget.value = bgr8_to_jpeg(frame)
        #cv2.imshow('Capture', frame)

        # change to hsv model
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        mask = cv2.inRange(hsv, color_lower, color_upper)
        #cv2.imshow('Mask', mask)
        mask_widget.value = bgr8_to_jpeg(mask)


        res = cv2.bitwise_and(frame, frame, mask=mask)
        #cv2.imshow('Result', res)
        result_widget.value = bgr8_to_jpeg(res)

        #     if cv2.waitKey(1) & 0xFF == ord('q'):
        #         break
        time.sleep(0.01)


    cap.release()
    #cv2.destroyAllWindows()

In [ ]:
thread1 = threading.Thread(target=Color_Recongnize)
thread1.setDaemon(True)
thread1.start()

In [ ]:
#End process, only need to execute this code at the end
# #结束进程，只有在结束时才需要执行此段代码
try:
    while True:
        time.sleep(.000001)
except KeyboardInterrupt:
    print(" Program closed! ")
    pass
stop_thread(thread1)

In [ ]:
cap.release()            